In [7]:
# Run setup from config notebook
%run 0_config_setup.ipynb

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


GPU Available: True
GPU Name: NVIDIA GeForce RTX 5090
GPU Memory: 33.67 GB
GPU Available: True
GPU Name: NVIDIA GeForce RTX 5090
GPU Memory: 33.67 GB
Using virtual environment at: /home/imane/gemma_env
Hugging Face token loaded from cache.
Directory structure and model paths are ready!
Configuration loaded successfully!
Utility functions loaded!
Configuration saved to /home/imane/Downloads/Reinforcement Learning/config.json
Using virtual environment at: /home/imane/gemma_env
Hugging Face token loaded from cache.
Directory structure and model paths are ready!
Configuration loaded successfully!
Utility functions loaded!
Configuration saved to /home/imane/Downloads/Reinforcement Learning/config.json


In [8]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm import tqdm
import json
import itertools
import random

set_seed(SEED)

## Load SFT Model

In [9]:
# Clear GPU cache if needed
import gc
import os

# Set CUDA environment variable for better error reporting
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

if torch.cuda.is_available():
    torch.cuda.empty_cache()
    gc.collect()
    print("✓ GPU cache cleared")

# Force CPU mode due to CUDA kernel incompatibility
FORCE_CPU = True  # Set to True to use CPU (RECOMMENDED due to kernel errors)
USE_8BIT = False  # Only relevant if FORCE_CPU=False
print(f"Device mode: {'CPU (forced)' if FORCE_CPU else ('GPU with 8-bit quantization' if USE_8BIT else 'GPU (full precision)')}")
print("⚠ Using CPU mode to avoid 'no kernel image' error")

✓ GPU cache cleared
Device mode: CPU (forced)
⚠ Using CPU mode to avoid 'no kernel image' error


In [10]:
print("Loading SFT model from Hugging Face...")

# Load directly from Hugging Face Hub
model_name = "ModelSpace/GemmaX2-28-9B-v0.1"

# Load with forced CPU mode if kernel incompatibility detected
if FORCE_CPU:
    print("⚠ Loading model on CPU (FORCE_CPU=True)")
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True, trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map={"": "cpu"},
        torch_dtype=torch.float32,
        trust_remote_code=True,
        use_auth_token=True,
        low_cpu_mem_usage=True
    )
    print("✓ Model loaded on CPU")
elif USE_8BIT:
    print("⚠ Loading model on GPU with 8-bit quantization")
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True, trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto",
        load_in_8bit=True,
        trust_remote_code=True,
        use_auth_token=True
    )
    print("✓ Model loaded on GPU with 8-bit quantization")
else:
    # Attempt to load on GPU (device_map="auto") and fall back to CPU if we hit CUDA/kernel errors
    try:
        tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True, trust_remote_code=True)
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.bfloat16,
            device_map="auto",
            use_auth_token=True,
            trust_remote_code=True
        )
        print("✓ Model loaded onto GPU/accelerator (device_map='auto').")
    except Exception as e:
        err = str(e)
        print("⚠ Warning: failed to load model on GPU/accelerator:", err)
        # Heuristics: handle CUDA driver / kernel mismatch or OOM by falling back to CPU
        if ("no kernel image" in err) or ("CUDA error" in err) or ("out of memory" in err) or ("failed to load" in err) or ("CUDA" in err and "kernel" in err):
            print("↪ Falling back to CPU to avoid CUDA/kernel incompatibility.")
            tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True, trust_remote_code=True)
            model = AutoModelForCausalLM.from_pretrained(
                model_name,
                device_map={"": "cpu"},
                torch_dtype=torch.float32,
                trust_remote_code=True,
                use_auth_token=True,
                low_cpu_mem_usage=True
            )
        else:
            # re-raise unexpected errors
            raise

# Ensure padding token is set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = tokenizer.eos_token_id

# Report device placement
try:
    device_summary = model.hf_device_map if hasattr(model, 'hf_device_map') else { '': str(next(model.parameters()).device) }
    print("Device placement:", device_summary)
except Exception:
    pass

print(f"Model size: {sum(p.numel() for p in model.parameters()) / 1e9:.2f}B parameters")
print("✓ Model ready for inference")

Loading SFT model from Hugging Face...
⚠ Loading model on CPU (FORCE_CPU=True)


/home/imane/gemma_env/lib/python3.12/site-packages/transformers/models/auto/tokenization_auto.py:1041: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/home/imane/gemma_env/lib/python3.12/site-packages/transformers/models/auto/auto_factory.py:492: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/home/imane/gemma_env/lib/python3.12/site-packages/transformers/models/auto/auto_factory.py:492: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!
`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 5/5 [00:20<00:00,  4.03s/it]



✓ Model loaded on CPU
Device placement: {'': 'cpu'}
Model size: 9.24B parameters
✓ Model ready for inference


## Load Training Data

In [11]:
# Load test prompts (NO parallel corpus needed - Phase 1 already complete!)
print("Loading test prompts for synthetic data generation...")
print("Note: We don't need reference translations - just source texts\n")

try:
    all_data = load_test_prompts(TEST_PROMPTS)
    print(f"Loaded {len(all_data)} test prompts from file")
except FileNotFoundError:
    print("Creating sample test prompts...")
    # Create sample prompts if file doesn't exist
    all_data = [
        {"source": "Hello, how are you?", "source_lang": "en"},
        {"source": "Good morning, have a nice day.", "source_lang": "en"},
        {"source": "Thank you for your help.", "source_lang": "en"},
        {"source": "Where is the nearest hospital?", "source_lang": "en"},
        {"source": "I would like to order some food.", "source_lang": "en"},
        {"source": "Bonjour, comment allez-vous?", "source_lang": "fr"},
        {"source": "Merci beaucoup pour votre aide.", "source_lang": "fr"},
        {"source": "Où se trouve la gare?", "source_lang": "fr"},
    ] * 100  # Replicate for testing
    print(f"Created {len(all_data)} sample prompts")

# Use a subset for synthetic data generation (adjust as needed)
SYNTHETIC_DATA_SIZE = min(5000, len(all_data))
random.shuffle(all_data)
training_samples = all_data[:SYNTHETIC_DATA_SIZE]

print(f"\nUsing {len(training_samples)} samples for synthetic data generation")

Loading test prompts for synthetic data generation...
Note: We don't need reference translations - just source texts

Loaded 91 test prompts from file

Using 91 samples for synthetic data generation


## Generate Translation Candidates

In [12]:
def generate_candidates(source_text, source_lang, num_candidates=8):
    """Generate multiple translation candidates with varying sampling parameters"""
    
    prompt = format_translation_prompt(source_text, source_lang)
    inputs = tokenizer(prompt, return_tensors="pt", padding=True).to(model.device)
    
    candidates = []
    
    # Generate candidates with different sampling strategies
    sampling_configs = []
    for temp in TEMPERATURES:
        for top_k in TOP_K_VALUES:
            for top_p in TOP_P_VALUES:
                sampling_configs.append({
                    'temperature': temp,
                    'top_k': top_k,
                    'top_p': top_p
                })
    
    # Sample configurations to get desired number of candidates
    selected_configs = random.sample(sampling_configs, min(num_candidates, len(sampling_configs)))
    
    for config in selected_configs:
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=256,
                temperature=config['temperature'],
                top_k=config['top_k'],
                top_p=config['top_p'],
                do_sample=True,
                num_return_sequences=1,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id
            )
        
        # Decode and extract translation
        full_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        # Extract only the generated translation (after the prompt)
        translation = full_text.split("Arabic translation:")[-1].strip()
        
        candidates.append({
            'translation': translation,
            'config': config
        })
    
    return candidates

# Test generation
test_sample = training_samples[0]
print(f"Testing with: {test_sample['source'][:100]}...")
test_candidates = generate_candidates(test_sample['source'], test_sample['source_lang'], num_candidates=8)
print(f"\nGenerated {len(test_candidates)} candidates:")
for i, cand in enumerate(test_candidates, 1):
    print(f"{i}. {cand['translation'][:100]}...")

Testing with: Whether organize files will group directories according to their filetype....

Generated 8 candidates:
1. سواء كانت تنظيم الملفات ستجمع الدلائل وفقًا لنوع الملف....
2. سوف تُعزز ملفات التنظيم المجلدات وفقًا لنوع ملفها....
3. سواء كانت تنظيم الملفات ستجمع الدلائل حسب نوع الملف....
4. سيتم جمع الملفات حسب نوع الملف....
5. سواء كان تصنيف الملفات سيتم فيه تجميع الدلائل وفقًا لنوع الملف....
6. سواءً كانت الملفات المنظمة ستُجمع الدلائل وفقًا لنوع ملفاتها....
7. سوف تمثل ملفات التنظيم المجموعات التجهيزية وفقاً لنوعها....
8. سوف يجمع تنظيم الملفات المجلدات وفقًا لنوع الملف الخاص بها....

Generated 8 candidates:
1. سواء كانت تنظيم الملفات ستجمع الدلائل وفقًا لنوع الملف....
2. سوف تُعزز ملفات التنظيم المجلدات وفقًا لنوع ملفها....
3. سواء كانت تنظيم الملفات ستجمع الدلائل حسب نوع الملف....
4. سيتم جمع الملفات حسب نوع الملف....
5. سواء كان تصنيف الملفات سيتم فيه تجميع الدلائل وفقًا لنوع الملف....
6. سواءً كانت الملفات المنظمة ستُجمع الدلائل وفقًا لنوع ملفاتها....
7. سوف تمثل ملفات الت

## Load Automatic Metrics

In [13]:
from comet import download_model, load_from_checkpoint
from bert_score import score as bert_score
from sacrebleu.metrics import CHRF

print("Loading evaluation metrics...")

# Load COMET model
comet_model_path = download_model("Unbabel/wmt22-comet-da")
comet_model = load_from_checkpoint(comet_model_path)
print("✓ COMET loaded")

# CHRF metric
chrf_metric = CHRF()
print("✓ CHRF loaded")

print("✓ BERTScore (will load on first use)")
print("\nAll metrics ready!")

Loading evaluation metrics...


Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 34267.19it/s]

Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.6.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../.cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/2760a223ac957f30acfb18c8aa649b01cf1d75f2/checkpoints/model.ckpt`
Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.6.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../.cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/2760a223ac957f30acfb18c8aa649b01cf1d75f2/checkpoints/model.ckpt`
Encoder model frozen.
Encoder model frozen.


✓ COMET loaded
✓ CHRF loaded
✓ BERTScore (will load on first use)

All metrics ready!


/home/imane/gemma_env/lib/python3.12/site-packages/pytorch_lightning/core/saving.py:197: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']


## Score Translations

In [14]:
def score_translation(source, translation):
    """Score a translation using reference-free metrics.

    Returns a dict with keys: 'comet', 'length_score', 'arabic_score', 'combined'.
    """
    scores = {}

    # COMET-QE (Quality Estimation - no reference needed)
    try:
        comet_input = [{
            'src': source,
            'mt': translation
        }]
        comet_output = comet_model.predict(comet_input, batch_size=1, gpus=1)
        scores['comet'] = float(comet_output.scores[0])
    except Exception as e:
        # If COMET fails, warn and use a fallback score of 0.0
        print("Warning: COMET scoring failed:", e)
        scores['comet'] = 0.0

    # Simple heuristic scores (no reference needed)
    src_len = len(source.split())
    tgt_len = len(translation.split())
    length_ratio = min(tgt_len, src_len * 2) / max(src_len, 1)  # Penalize very long/short
    scores['length_score'] = float(min(length_ratio, 1.0))

    # 2. Contains Arabic characters (basic validation)
    arabic_chars = sum(1 for c in translation if '\u0600' <= c <= '\u06FF')
    scores['arabic_score'] = float(min(arabic_chars / max(tgt_len, 1), 1.0))

    # Weighted combined score
    combined_score = (
        scores['comet'] * 0.7 +  # Main quality metric
        scores['length_score'] * 0.2 +
        scores['arabic_score'] * 0.1
    )
    scores['combined'] = float(combined_score)

    return scores

# Quick self-test (prints scores for the first test candidate if available)
try:
    test_scores = score_translation(test_sample['source'], test_candidates[0]['translation'])
    print("Scores:", {k: f"{v:.4f}" for k, v in test_scores.items()})
except Exception as e:
    print("Test scoring failed:", e)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
You are using a CUDA device ('NVIDIA GeForce RTX 5090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
You are using a CUDA device ('NVIDIA GeForce RTX 5090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For

Search for `cudaErrorNoKernelImageForDevice' in https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__TYPES.html for more information.
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.

Scores: {'comet': '0.0000', 'length_score': '0.9000', 'arabic_score': '1.0000', 'combined': '0.2800'}


## Generate Synthetic Preference Dataset

In [ ]:
def create_preference_pairs(candidates_with_scores):
    """Create pairwise preference data from scored candidates"""

    preference_pairs = []

    # Sort candidates by combined score
    sorted_candidates = sorted(candidates_with_scores, key=lambda x: x['scores']['combined'], reverse=True)

    # Create pairs: better vs worse
    # Compare each candidate with worse ones
    for i in range(len(sorted_candidates)):
        for j in range(i + 1, len(sorted_candidates)):
            # i has higher score than j
            chosen = sorted_candidates[i]
            rejected = sorted_candidates[j]

            # Only create pair if there's a meaningful difference
            score_diff = chosen['scores']['combined'] - rejected['scores']['combined']
            if score_diff > 0.05:  # Minimum threshold for meaningful difference
                preference_pairs.append({
                    'chosen': chosen['translation'],
                    'rejected': rejected['translation'],
                    'chosen_score': chosen['scores']['combined'],
                    'rejected_score': rejected['scores']['combined'],
                    'score_margin': score_diff
                })

    return preference_pairs

print("Generating synthetic preference dataset...")
print(f"Processing {len(training_samples)} samples...\n")

synthetic_dataset = []

# Path to save all generated candidates per source for inspection
candidates_path = DATA_DIR / "generated_candidates.jsonl"
print(f"Saving generated candidates to {candidates_path}")

# Open file once and append per sample
with open(candidates_path, 'w', encoding='utf-8') as cand_f:
    for idx, sample in enumerate(tqdm(training_samples, desc="Generating preferences")):
        try:
            # Generate candidates
            candidates = generate_candidates(
                sample['source'],
                sample['source_lang'],
                num_candidates=NUM_CANDIDATES
            )

            # Save raw candidates for this sample
            cand_record = {
                'source': sample['source'],
                'source_lang': sample['source_lang'],
                'candidates': candidates
            }
            cand_f.write(json.dumps(cand_record, ensure_ascii=False) + '\n')
            cand_f.flush()

            # Score each candidate (no reference needed)
            scored_candidates = []
            for cand in candidates:
                scores = score_translation(
                    sample['source'],
                    cand['translation']
                )
                scored_candidates.append({
                    'translation': cand['translation'],
                    'scores': scores,
                    'config': cand['config']
                })

            # Create preference pairs
            preference_pairs = create_preference_pairs(scored_candidates)

            # Store with metadata
            for pair in preference_pairs:
                synthetic_dataset.append({
                    'source': sample['source'],
                    'source_lang': sample['source_lang'],
                    'chosen': pair['chosen'],
                    'rejected': pair['rejected'],
                    'chosen_score': pair['chosen_score'],
                    'rejected_score': pair['rejected_score'],
                    'margin': pair['score_margin']
                })
        except Exception as e:
            print(f"Error processing sample {idx}: {e}")
            continue

        # Progress update every 100 samples
        if (idx + 1) % 100 == 0:
            print(f"\nProcessed {idx + 1}/{len(training_samples)} samples")
            print(f"Generated {len(synthetic_dataset)} preference pairs so far\n")

print(f"\n✓ Generation complete!")
print(f"Total preference pairs: {len(synthetic_dataset)}")

Generating synthetic preference dataset...
Processing 91 samples...

Saving generated candidates to /home/imane/Downloads/Reinforcement Learning/data/generated_candidates.jsonl


Generating preferences:   0%|          | 0/91 [00:00<?, ?it/s]💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU c

Search for `cudaErrorNoKernelImageForDevice' in https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__TYPES.html for more information.
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.



💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Search for `cudaErrorNoKernelImageForDevice' in https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__TYPES.html for more information.
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.



...

Search for `cudaErrorNoKernelImageForDevice' in https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__TYPES.html for more information.
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.



💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Search for `cudaErrorNoKernelImageForDevice' in https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__TYPES.html for more information.
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.



Generating preferences:  21%|██        | 19/91 [59:05<3:10:50, 159.04s/it]

Search for `cudaErrorNoKernelImageForDevice' in https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__TYPES.html for more information.
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.



## Save Synthetic Dataset

In [ ]:
# Save to JSONL format
print(f"Saving dataset to {SYNTHETIC_PREFERENCES}...")

with open(SYNTHETIC_PREFERENCES, 'w', encoding='utf-8') as f:
    for item in synthetic_dataset:
        f.write(json.dumps(item, ensure_ascii=False) + '\n')

print(f"✓ Saved {len(synthetic_dataset)} preference pairs")

# Save summary statistics
stats = {
    'total_pairs': len(synthetic_dataset),
    'avg_margin': sum(item['margin'] for item in synthetic_dataset) / len(synthetic_dataset),
    'avg_chosen_score': sum(item['chosen_score'] for item in synthetic_dataset) / len(synthetic_dataset),
    'avg_rejected_score': sum(item['rejected_score'] for item in synthetic_dataset) / len(synthetic_dataset),
    'num_sources': len(training_samples)
}

stats_path = DATA_DIR / "synthetic_data_stats.json"
with open(stats_path, 'w') as f:
    json.dump(stats, f, indent=2)

print(f"\nDataset Statistics:")
for key, value in stats.items():
    print(f"  {key}: {value if isinstance(value, int) else f'{value:.4f}'}")

## Sample Preference Pairs

In [ ]:
# Display some examples
print("Sample preference pairs:\n")
print("=" * 80)

for i, item in enumerate(random.sample(synthetic_dataset, min(3, len(synthetic_dataset))), 1):
    print(f"\nExample {i}:")
    print(f"Source ({item['source_lang']}): {item['source'][:150]}")
    print(f"\nChosen (score: {item['chosen_score']:.3f}): {item['chosen'][:150]}")
    print(f"\nRejected (score: {item['rejected_score']:.3f}): {item['rejected'][:150]}")
    print(f"\nMargin: {item['margin']:.3f}")
    print("=" * 80)

## Next Step

Proceed to **notebook 2** to train the reward model using this synthetic preference data.

## Quick Process: Convert Partial Data to Preferences

Run this cell to process the candidates you already generated (without re-running generation).